In [ ]:
"""Convert old eq to new one"""
#rate =(shift+j)/k)*p
#return (rate*l)/m
    
j = 1.7089
k = 2.2418
l = 70
m = 10
p = 1   # 1 or a percentage

a = l/m
b = p/k
c = j/k*p

print('a =', a)
print('b =', round(b, 4))
print('c =', round(c, 4))

In [ ]:
"""Setup for Sci Trim Calculations:
Please only edit in designated area,
Will return found file name and list of all colums in file (use to select target_param)
"""
# Stdlib imports
import os
import sys
import copy
import csv
from functools import partial

# Third-Party imports
import pandas as pd
import numpy as np
from scipy.interpolate import griddata
from numpy import mgrid
# make MPL inteactiveable
%matplotlib notebook
import matplotlib.pyplot as plt
# interactive widgets
import ipywidgets as widgets
from IPython.display import display

# make modules avaiable from local
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
# Local imports
from fpy.file_funcs import find_base_path, find_test_path, get_filename, get_target_param, read_waferview, read_ibe, write_ibe, file_not_exist
from fpy.trim_funcs import trimToThickness, trimToFreq
from fpy.basic_tools import filt_gt, filt_3sigma
from fpy.plot_tools import AnnoteFinder, plot_scatter, FitLinear, plot_wafermap, plot_wafer



# Edit These!! ################################################################
freq          = "1090"
wafer         = 10219
d8_num        = "0015"
#test_step     = "ResonatorMap"  
test_step     = "WaferMap"

# Leave these as "" to auto find 
test_spec     = ""  # "m2-d8", "d8", "m3-d8",  "d8", or "dwg-"
target_param  = "" #"ResFreq"
#target_param  = "ResFreq"  # 'ResFreq(S11)(1)', 'Vswr(-3-0)(S11)(1)', 'CF10(S21)(1)'
file_num      = -2 # "08", -1 finds most recient
comp_fn       = -3 # "06", -2 finds second most recient
rm_param      = ''
rm_gt         = 8  # remove rows with rm_param > rm_gt
rm_lt         = None  # remove rows with rm_param < rm_lt

# Set to None if not using
trim_to_freq = 1662
trim_to_thick = None

# rate eq
# removal[nm] = a[nm/s] * (shift[MHz] * b[s/MHz] + c[s])

a = 2.1
b = 1.3333
c = 0

# Don't edit past here!! #######################################################

def rate_func_base(shift=None, a=None, b=None, c=None):

    """thickness_to_trim[nm] = rate_func(trim_to_freq - actual_freq)
    Standard form is:
        rate = (shift+X)/Y
        return (rate*z)/10
    Must return a value in nm,
    If trimming to vwsr where shift = (actual-trim_to) just make shift -: rate = (-shift...)
    """
    removal = a*(shift*b + c)
    return removal 

rate_func = partial(rate_func_base, a=a, b=b, c=c)

base_path = find_base_path()
full_path = find_test_path(base_path, str(freq), str(wafer), test_step, d8_num, test_spec)
path = os.path.join(full_path, get_filename(full_path, file_num))
print(path)
test_s = read_waferview(path, rm_param, rm_gt, rm_lt)
"""Show list of all columns"""
print("\n Use this list to detrimine what to put in 'target_param':")
display(list(test_s.dataframe))
target_param = get_target_param(test_s, test_step, target_param)

# get test to compair too and ibe
target = float(trim_to_freq)
comp_fn = get_filename(full_path, comp_fn)
if comp_fn:
    comp_path = os.path.join(full_path, comp_fn)
    test_comp_s = read_waferview(comp_path, rm_param, rm_gt, rm_lt)
    ibe_s = read_ibe(comp_path)
    if ibe_s.header:
        target = float(ibe_s.header['target'][0:-4])
    test_s.dataframe['target_shift'] = target - test_comp_s.dataframe[target_param]
    test_s.dataframe['actual_shift'] = test_s.dataframe[target_param] - test_comp_s.dataframe[target_param]
    test_s.dataframe['diff_shift'] = test_s.dataframe['target_shift'] - test_s.dataframe['actual_shift']

print('target:', target)
#"""Show stats for all columns"""
#stats = test_s.dataframe.describe()
#display(stats)

In [ ]:
"""Make new ibe based on parameters set above"""
if trim_to_freq:
    target = str(trim_to_freq) +" MHz"
    ibe_filt = trimToFreq(trim_to_freq, target_param, test_s.dataframe, test_s.header['FlatLocation'], rate_func)
if trim_to_thick:
    target = str(trim_to_thick) + " nm"
    ibe_filt = trimToThickness(trim_to_thick, test_s.dataframe)    
# Write the ibe
if file_not_exist(path + '.ibe'):
    write_ibe(path + '.ibe', ibe_filt, target, [a, b, c])  # save backup
    write_ibe(os.path.join(full_path, str(wafer) + '.ibe'), ibe_filt, target, [a, b, c])  # save to run on scia
    
display(ibe_filt)

In [ ]:
"""Show stats for target_param"""
print('Current')
stats = filt_gt(test_s.dataframe[[target_param]], [0]).describe()
display(stats)
print('target:', target, '\n')
print('Previous')
display(filt_gt(test_comp_s.dataframe[[target_param]], [0]).describe())


In [ ]:
"""Plot change"""
df = test_s.dataframe[['target_shift', 'actual_shift']].copy()
x_nm = 'target_shift'
y_nm = 'actual_shift'

# remove < 0
dfc = filt_gt(df, [0, 1])
print('Total removed < 0:', df.shape[0] - dfc.shape[0], 'of', df.shape[0])
# remove > 3 sigma
dfd = filt_3sigma(dfc, [0, 1])
print('Total removed > 3\u03c3:', dfc.shape[0] - dfd.shape[0], 'of', df.shape[0])
xx = dfd[x_nm]
yy = dfd[y_nm]

# extra filtering
#filt = xx < 25
#filt = [a or b for a, b in zip(yy > 10, yy < 5)]
#filt = yy < 6
#xx = xx[filt].reset_index(drop=True)
#yy = yy[filt].reset_index(drop=True)
#################

a = float(ibe_s.header['a'])
b = float(ibe_s.header['b'])
c = float(ibe_s.header['c'])

print("a =",a ,"\nb =",b ,"\nc =",c ,'\n')
fig, ax = plt.subplots(figsize=(5,5))
ax = plot_scatter(xx, yy, x_nm, y_nm, ax)
# Make points selectable
af =  AnnoteFinder(xx, yy, ax=ax)
fig.canvas.mpl_connect('button_press_event', af)


# Make fit
fit = FitLinear(xx, yy, ax=ax)

fig.tight_layout()

display('Please Select the points to remove from analysis then refit:')
# Make button for refitting process
refit_button = widgets.Button(description="Refit")
display(refit_button)
# Make button to display fit and new eq
geteq_button = widgets.Button(description="Get New Equation")
display(geteq_button)

def refit_button_clicked(button):
    """refit data removing points selected in af"""
    # Make deepcopies to maintain data integrity
    p_sort = copy.deepcopy(af.points)
    xa = copy.deepcopy(xx)
    ya = copy.deepcopy(yy)
    # sort found points in decending order so pop works right then pop(remove indices)
    p_sort.sort(key=lambda a: a[0], reverse=True)
    for point in p_sort:
        xa.pop(point[0])
        ya.pop(point[0])
    # Reset the indices (needed to plot correctly)    
    xa = xa.reset_index(drop=True)
    ya = ya.reset_index(drop=True)
    # Update the fit
    fit.update_fit(xa, ya)

def geteq_button_clicked(button):
    """Get new equation for trimming based on fit.
    takes initial eq and matchs new removal form previous run."""
    # get slope and intercept from fit
    m = fit.slope
    n = fit.intercept
    # removal[nm] = a[nm/s] * (shift[MHz] * b[s/MHz] + c[s])
    # actual vs target line fite eq y=mx+n
    b2 = b/m # + 1/m/a
    if n > 0:
        c2 = c - n*b2
    else:
        c2 = c - n/a*b2
    print('\nNew Eq:')
    print('a =', a)
    print('b =', round(b2, 4))
    print('c =', round(c2, 4))

# Connect buttons
refit_button.on_click(refit_button_clicked)
geteq_button.on_click(geteq_button_clicked)

In [ ]:
"""Some Nice Data Plots"""

fig, ax = plt.subplots(figsize=(7,5))
#plot x, y dependancy
ax.plot([-40000, 40000],[target, target])
#x='age', y='fdg', marker='.'
dfs = [test_s.dataframe[['DieX', 'DieY', target_param]].copy(), test_comp_s.dataframe[['DieX', 'DieY', target_param]].copy()]

colors = [['r', 'g'],['b', 'y']]
names = [['current_X', 'previous_X'],['current_Y', 'previous_Y']]
for ii, df in enumerate(dfs):
    """Plot 2D"""
    filt_3sigma(filt_gt(df, [1]), [0, 1, 2]).plot(x='DieX', y=target_param, ax=ax, kind='scatter', color=colors[ii][0], label=names[ii][0])
    filt_3sigma(filt_gt(df, [2]), [0, 1, 2]).plot(x='DieY', y=target_param, ax=ax, kind='scatter', color=colors[ii][1], label=names[ii][1])
ax.set_xlabel('Die Position [\u03BCm]')
ax.legend(bbox_to_anchor=(1.1, 1.05))
fig.tight_layout()

"""Plot Wafer Maps"""
# set up axis
fig, ((ax11, ax12, ax13,), (ax21, ax22, ax23)) = plt.subplots(2, 3, figsize=(9.5, 5))

bins = 11
df11 = filt_3sigma(filt_gt(test_s.dataframe[['DieX', 'DieY', target_param]].copy(), [2]), [0, 1, 2])
df12 = filt_3sigma(filt_gt(test_comp_s.dataframe[['DieX', 'DieY', target_param]].copy(), [2]), [0, 1, 2])
df13 = filt_3sigma(filt_gt(test_s.dataframe[['DieX', 'DieY', 'diff_shift']].copy(), []), [0, 1, 2])
df21 = filt_3sigma(filt_gt(test_s.dataframe[['DieX', 'DieY', 'target_shift']].copy(), [2]), [0, 1, 2])
df22 = filt_3sigma(filt_gt(test_s.dataframe[['DieX', 'DieY', 'actual_shift']].copy(), [2]), [0, 1, 2])    
df23 = ibe_s.dataframe

# max and min for 11 and 12
vmin = round(np.min(df12[target_param]))
vmax = round(np.max(df11[target_param]))
print(vmin, vmax)
# plot current
plot_wafermap(ax11, df11['DieX'], df11['DieY'], df11[target_param], flatlocal=float(test_s.header['FlatLocation']), bins=bins,)# vmin=vmin, vmax=vmax, )
ax11.set_title('Current')
# plot_previous
plot_wafermap(ax12, df12['DieX'], df12['DieY'], df12[target_param], flatlocal=float(test_comp_s.header['FlatLocation']), vmin=vmin, vmax=vmax, bins=bins)
ax12.set_title('Previous')
# plot diff shift
plot_wafermap(ax13, df13['DieX'], df13['DieY'], df13['diff_shift'], flatlocal=float(test_comp_s.header['FlatLocation']), bins=bins)
ax13.set_title('Diffrence')

# max and min for 21 and 22
vmin = round(min(np.min(df21['target_shift']), np.min(df22['actual_shift'])), 2)
vmax = round(max(np.max(df21['target_shift']), np.max(df22['actual_shift'])), 2)
# plot_target
plot_wafermap(ax21, df21['DieX'], df21['DieY'], df21['target_shift'], flatlocal=float(test_s.header['FlatLocation']), vmin=vmin, vmax=vmax, bins=bins)
ax21.set_title('Target')
# plot_actual
plot_wafermap(ax22, df22['DieX'], df22['DieY'], df22['actual_shift'], flatlocal=float(test_comp_s.header['FlatLocation']), vmin=vmin, vmax=vmax, bins=bins)
ax22.set_title('Actual')
# plot ibe
if float(test_comp_s.header['FlatLocation']) == 180:
    plot_wafermap(ax23, df23['y']*1000, -df23['x']*1000, df23['removal'], flatlocal=180, bins=bins)
    ax23.set_title('ibe')
elif float(test_comp_s.header['FlatLocation']) == 270:
    plot_wafermap(ax23, df23['x']*1000, df23['y']*1000, df23['removal'], flatlocal=270, bins=bins)
    ax23.set_title('ibe')
else:
    raise ValueError('Show Austin')
print(test_comp_s.header['FlatLocation'])
fig.tight_layout()

In [ ]:
"""Plot shift diff vs die x"""

yd = test_s.dataframe['diff_shift']
yb = yd<1000
yd = yd[yb].reset_index(drop=True)
xd = test_s.dataframe['DieX'][yb].reset_index(drop=True)
xe = test_s.dataframe['DieY'][yb].reset_index(drop=True)
xc = test_s.dataframe['target_shift'][yb].reset_index(drop=True)
print("a =",a ,"\nb =",b ,"\nc =",c ,'\n')
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(5,15))
ax1 = plot_scatter(xd, yd, 'dieX', 'diff shift', ax1)
fit1 = FitLinear(xd, yd, ax=ax1)
ax2 = plot_scatter(xe, yd, 'dieY', 'diff shift', ax2)
fit2 = FitLinear(xe, yd, ax=ax2)
ax3 = plot_scatter(xc, yd, 'target_shift', 'diff shift', ax3)
fit3 = FitLinear(xc, yd, ax=ax3)

fig.tight_layout()

In [ ]:
"""Correlation of data"""

corr = test_s.dataframe.corr()

fig, ax = plt.subplots(figsize=(15,15))
im = ax.matshow(corr, cmap=plt.get_cmap('seismic'))
plt.colorbar(im)
plt.xticks(range(len(corr.columns)), corr.columns, rotation=90)
plt.yticks(range(len(corr.columns)), corr.columns)

display(corr)

In [ ]:
"""Plot Wafer Map"""
df = test_s.dataframe
x = df['DieX']
y = df['DieY']
z = df['ResFreq(S11)(1)']

xq, yq = mgrid[-50000:50000:100j,-50000:50000:100j]
points = np.array([-y.T,x.T])
z.shape

Z = griddata(points.T, z, (xq,yq), method='cubic')
#h = plt.pcolormesh(xq,yq,Z)
# Draw a Wafer
circle = plt.Circle((0,0), 50000, fill=False)

fig, ax = plt.subplots()
ax.add_artist(circle)

g = ax.contourf(xq,yq,Z)
plt.colorbar(g)
#plt.clabel(g, inline=1, fontsize=10)

In [ ]:
"""Plot Thickness Map"""
df = test_s.dataframe
x = df['Die x (mm)']
y = df['Die y (mm)']
z = df['Site 1 Layer 1 Thickness (A)']
gof = df['Site 1 GOF']

xq, yq = mgrid[-50:50:100j,-50:50:100j]
points = np.array([x,y])

Z = griddata(points.T, z, (xq,yq), method='cubic')

#h = plt.pcolormesh(xq,yq,Z)
# Draw a Wafer
circle = plt.Circle((0,0), 50, fill=False)

fig, ax = plt.subplots()
ax.add_artist(circle)

g = ax.contourf(xq,yq,Z)
plt.colorbar(g)
#plt.clabel(g, inline=1, fontsize=10)